In [ ]:
import numpy as np
import pandas as pd
from numpy import linspace
import math

In [ ]:
import os
from os import path
from os import system

In [ ]:
def replaceZeroes(data):
    min_nonzero = np.min(data[np.nonzero(data)])
    data[data == 0] = min_nonzero
    return data

In [ ]:
LINE = 20

In [ ]:
def read_scalar(filename):
    # Read file
    file = open(filename,'r')
    lines_1 = file.readlines()
    file.close()

    num_cells_internal = int(lines_1[LINE].strip('\n'))
    lines_1 = lines_1[LINE+2:LINE+2+num_cells_internal]

    for i in range(len(lines_1)):
        lines_1[i] = lines_1[i].strip('\n')

    field = np.asarray(lines_1).astype('double').reshape(num_cells_internal,1)
    field = replaceZeroes(field)

    return field

In [ ]:
def read_vector(filename): # Only x,y components
    file = open(filename,'r')
    lines_1 = file.readlines()
    file.close()

    num_cells_internal = int(lines_1[LINE].strip('\n'))
    lines_1 = lines_1[LINE+2:LINE+2+num_cells_internal]

    for i in range(len(lines_1)):
        lines_1[i] = lines_1[i].strip('\n')
        lines_1[i] = lines_1[i].strip('(')
        lines_1[i] = lines_1[i].strip(')')
        lines_1[i] = lines_1[i].split()

    field = np.asarray(lines_1).astype('double')[:,:2]

    return field

In [ ]:
def read_mean_vector(filename): # Only x,y components
    file = open(filename,'r')
    lines_1 = file.readlines()
    file.close()

    num_cells_internal = int(lines_1[LINE].strip('\n'))
    lines_1 = lines_1[LINE+2:LINE+2+num_cells_internal]

    for i in range(len(lines_1)):
        lines_1[i] = lines_1[i].strip('\n')
        lines_1[i] = lines_1[i].strip('(')
        lines_1[i] = lines_1[i].strip(')')
        lines_1[i] = lines_1[i].split()

    field = np.asarray(lines_1).astype('double')[:,:3]

    return field

In [ ]:
def read_symmTensor(filename): # Only x,y components
    file = open(filename,'r')
    lines_1 = file.readlines()
    file.close()

    num_cells_internal = int(lines_1[LINE].strip('\n'))
    lines_1 = lines_1[LINE+2:LINE+2+num_cells_internal]

    for i in range(len(lines_1)):
        lines_1[i] = lines_1[i].strip('\n')
        lines_1[i] = lines_1[i].strip('(')
        lines_1[i] = lines_1[i].strip(')')
        lines_1[i] = lines_1[i].split()

    field = np.asarray(lines_1).astype('double')[:,3:4]

    return field

In [ ]:
def read_stress_vector(filename): # Only x,y components
    file = open(filename,'r')
    lines_1 = file.readlines()
    file.close()

    num_cells_internal = int(lines_1[LINE].strip('\n'))
    lines_1 = lines_1[LINE+2:LINE+2+num_cells_internal]

    for i in range(len(lines_1)):
        lines_1[i] = lines_1[i].strip('\n')
        lines_1[i] = lines_1[i].strip('(')
        lines_1[i] = lines_1[i].strip(')')
        lines_1[i] = lines_1[i].split()

    field = np.asarray(lines_1).astype('double')[:,:6]

    return field

In [ ]:
def find_max(path,filename):
    path = os.chdir(path)
    with open(filename, 'r') as inF:
        for line in inF:
            if 'SIMPLE solution converged in' in line:
                print ("true")
                temp   = line.rsplit()
                number = temp[4]  
    return number

In [ ]:
def find_flap_aoa(foldername):
    temp = foldername.split('_')
    print(temp)
    flap,aoa = int(temp[3]),int(temp[5])
    flap = math.radians(flap) 
    aoa = math.radians(aoa) 
    return flap,aoa

In [ ]:
def read_data(path):
    path = os.chdir(path)
    U = read_vector(f'U')
    C = read_vector(f'C')
    Umean = read_vector(f'UMean')
    nut = read_scalar(f'nut')

    temp_dataset = np.concatenate((U,C,Umean,nut),axis=-1)

    type(temp_dataset)
    
    return temp_dataset
    


In [ ]:
def find_factor(list_path):
#     path = os.chdir((os.path.join(list_path,'system')))
    path = os.chdir((os.path.join(list_path,'system')))
    print(path)
    with open('blockMeshDict', 'r') as inF:
        for line in inF:
            if 'factor1' in line:
                print ("true")
                temp   = line.rsplit()
                factor1 = (temp[1])
                factor1 = float(factor1.replace(";", ""))
                
            if 'factor2' in line:
                print ("true")
                temp   = line.rsplit()
                factor2 = (temp[1])
                factor2 = float(factor2.replace(";", ""))
                
    
    return factor1,factor2

In [ ]:
if __name__ == '__main__':
    print('Velocity reader file')
    
temp_dataset = []
total_dataset = []
dir_path = os.path.abspath('## local path of openfoam runs ##')
print(dir_path)

folders = os.listdir(dir_path)

if os.path.isdir(dir_path): # Check this path = path to folder
    for folder in folders:
        print(folders)
        if folder !=  '.ipynb_checkpoints':
            flap,aoa = find_flap_aoa(folder)
            print(flap)
            print(aoa)
            print('-------------------------------')
            sub_folders = os.listdir(os.path.join(dir_path,folder))
            if os.path.isdir(os.path.join(dir_path,folder)): # Check this path = path to folder
                print("the sub folder are:",sub_folders)
                print('--------------------------------')

                   
            for each_folder in sub_folders:
                print(each_folder)
                list_path = os.path.join(dir_path,folder,each_folder)
                os.path.join(list_path)                                  
                if os.path.isdir(list_path): # Check this path = path to folder
                    list_folders = os.listdir(list_path)
                    print (list_folders)
                    print(list_path)
                    max_folder = find_max(list_path,f'log')
                    print(max_folder)
                    file_path = os.path.join(list_path,max_folder)
                    list_files = os.listdir(os.path.join(list_path,max_folder))
                    temp_data = read_data(file_path)
                    print(temp_data)
                    print(temp_data.shape)
                temp_dataset.append(temp_data)


#             total_dataset.append(temp_data)

total_dataset = np.reshape(temp_dataset, (-1,7))

print(total_dataset.shape)

# Save data  
path = os.chdir(dir_path)
np.save('Total_dataset.npy',total_dataset)

# Save the statistics of the data
means = np.mean(total_dataset,axis=0).reshape(1,np.shape(total_dataset)[1])
stds = np.std(total_dataset,axis=0).reshape(1,np.shape(total_dataset)[1])

# Concatenate
op_data = np.concatenate((means,stds),axis=0)
np.savetxt('means',op_data, delimiter=' ')

# Need to write out in OpenFOAM rectangular matrix format

print('Means:')
print(means)
print('Stds:')
print(stds)

                